In [1]:
#importando bibliotecas
import pandas as pd
import folium
from folium import plugins


In [3]:
#Fazendo a leitura do arquivo csv, alterando a configuração de separação e codificação do texto
df =pd.read_csv('./data/Bahia_Book1.csv', encoding='ISO-8859-1') #UTF-8 /cp1252

df.head()
#df.head()

,state,city,SUM of last_available_confirmed,SUM of new_confirmed_7day_rolling,SUM of last_available_deaths,SUM of new_deaths_7day_rolling,MIN of date,MIN of estimated_population_2019
0,BA,AbaÌ?ra,276,2,11,0,6/15/2021,8739
1,BA,AbarÌ©,919,5,11,0,6/15/2021,20086
2,BA,Acajutiba,500,1,16,0,6/15/2021,15159
3,BA,Adustina,869,7,20,0,6/15/2021,17040
4,BA,Ìgua Fria,168,2,10,0,6/15/2021,16970


In [3]:
#Limpando todas as colunas que não interessam para o projeto:
df = df.drop('state', 1)
df = df.drop('SUM of new_confirmed_7day_rolling', 1)
df = df.drop('SUM of last_available_deaths', 1)
#df_coord = df_coord.drop('nome', 1)
df = df.drop('SUM of new_deaths_7day_rolling', 1)
df = df.drop('MIN of date', 1)
#df = df.drop('MIN of estimated_population_2019', 1)
df = df.drop('city', 1)

C:\Users\jsilv757\AppData\Local\Temp/ipykernel_4256/1385492578.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('state', 1)
C:\Users\jsilv757\AppData\Local\Temp/ipykernel_4256/1385492578.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('SUM of new_confirmed_7day_rolling', 1)
C:\Users\jsilv757\AppData\Local\Temp/ipykernel_4256/1385492578.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('SUM of last_available_deaths', 1)
C:\Users\jsilv757\AppData\Local\Temp/ipykernel_4256/1385492578.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('SUM of new_deaths_7day_rolling', 1)
C:\Users\jsilv75

In [4]:
#verificar quais as cidades no dataset
df.shape

(417, 2)

In [5]:
#Mudar o nome da coluna dos casos pra ficar mais fácil a manipulação
df.rename(columns={'casos': 'SUM of last_available_confirmed', 
                           'SUM of last_available_confirmed': 'casos'}, inplace=True)

#Mudar o nome da coluna da população pra ficar mais fácil a manipulação
df.rename(columns={'populacao': 'MIN of estimated_population_2019', 
                           'MIN of estimated_population_2019': 'populacao'}, inplace=True)

df.head()

,casos,populacao
0,276,8739
1,919,20086
2,500,15159
3,869,17040
4,168,16970


Já que o nosso dataset de casos não tem as coordenadas (Latitude e Longitude) de cada cidade, vamos importar outro dataset com as coordenadas das cidades da bahia:

In [6]:
df_coord = pd.read_csv('./data/municipios.csv')
df_coord.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [7]:
#De acordo com a documentação do dataset o codigo UF da bahia é 29

#Limpar de todos os estados menos da bahia do dataset
indexNums = df_coord[ df_coord['codigo_uf'] != 29 ].index
df_coord.drop(indexNums , inplace=True)

#resetar index
df_coord.reset_index(inplace=True)

df_coord.head()

,index,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,6,2900108,Abaíra,-13.24880,-41.6619,0,29,3301,77,America/Sao_Paulo
1,7,2900207,Abaré,-8.72073,-39.1162,0,29,3303,75,America/Sao_Paulo
2,17,2900306,Acajutiba,-11.65750,-38.0197,0,29,3305,75,America/Sao_Paulo
3,34,2900355,Adustina,-10.54370,-38.1113,0,29,3253,75,America/Sao_Paulo
4,56,2900405,Água Fria,-11.86180,-38.7639,0,29,3307,75,America/Sao_Paulo


In [8]:
#Limpando todas as colunas que não interessam para o projeto:
df_coord = df_coord.drop('codigo_ibge', 1)
df_coord = df_coord.drop('index', 1)
df_coord = df_coord.drop('capital', 1)
#df_coord = df_coord.drop('nome', 1)
df_coord = df_coord.drop('siafi_id', 1)
df_coord = df_coord.drop('ddd', 1)
df_coord = df_coord.drop('fuso_horario', 1)

df_coord.head()

C:\Users\jsilv757\AppData\Local\Temp/ipykernel_4256/3576663500.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_coord = df_coord.drop('codigo_ibge', 1)
C:\Users\jsilv757\AppData\Local\Temp/ipykernel_4256/3576663500.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_coord = df_coord.drop('index', 1)
C:\Users\jsilv757\AppData\Local\Temp/ipykernel_4256/3576663500.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_coord = df_coord.drop('capital', 1)
C:\Users\jsilv757\AppData\Local\Temp/ipykernel_4256/3576663500.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_coord = df_coord.drop('siafi_id', 1)
C:\Users\jsilv757\AppData\Local\T

,nome,latitude,longitude,codigo_uf
0,Abaíra,-13.24880,-41.6619,29
1,Abaré,-8.72073,-39.1162,29
2,Acajutiba,-11.65750,-38.0197,29
3,Adustina,-10.54370,-38.1113,29
4,Água Fria,-11.86180,-38.7639,29


In [9]:
#checar tamanho pra verificar se o numero de cidades dos dois dataframes é igual
df_coord.shape

(417, 4)

Agora precisamos unir os dois dataframes em um só

In [10]:
#Juntando os dois dataframes:
df_res =  pd.concat([df_coord, df], axis=1, join='outer')
df_res

,nome,latitude,longitude,codigo_uf,casos,populacao
0,Abaíra,-13.24880,-41.6619,29,276,8739
1,Abaré,-8.72073,-39.1162,29,919,20086
2,Acajutiba,-11.65750,-38.0197,29,500,15159
3,Adustina,-10.54370,-38.1113,29,869,17040
4,Água Fria,-11.86180,-38.7639,29,168,16970
...,...,...,...,...,...,...
412,Vitória da Conquista,-14.86150,-40.8442,29,31424,338480
413,Wagner,-12.28190,-41.1715,29,814,9345
414,Wanderley,-12.11440,-43.8958,29,797,12238
415,Wenceslau Guimarães,-13.69080,-39.4762,29,1605,21101


In [11]:
#armazenando os dados do dataset
coordenadas=[]
for lat,lng,qnt in zip(df_res.latitude.values,df_res.longitude.values, df_res.casos):
  coordenadas.append([lat,lng,qnt])

#renderizando mapa
mapa = folium.Map(location=[-12.964152, -38.505821	],zoom_start=6.5,tiles='Stamen Toner')

#plotando mapa
mapa.add_child(plugins.HeatMap(coordenadas))        
mapa

In [12]:
#Salvar mapa
mapa.save('mapa-covid-bahia.html')


In [13]:
#fazer mapa de população
#armazenando os dados do dataset
coordenadas2=[]
for lat,lng,qnt2 in zip(df_res.latitude.values,df_res.longitude.values, df_res.populacao):
  coordenadas.append([lat,lng,qnt2])

#renderizando mapa
mapa2 = folium.Map(location=[-12.964152, -38.505821	],zoom_start=6.5,tiles='Stamen Toner')

#plotando mapa
mapa2.add_child(plugins.HeatMap(coordenadas))        
mapa2

In [ ]:
mapa2.save('mapa-populacao-bahia.html')